In [1]:
from preprocessing import preprocessing_data
import os
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

In [ ]:
# Original data

retards_df = pd.read_csv("data/regularite-mensuelle-tgv-aqst.csv", sep=';')
#retards_df.iloc[1]
retards_df["date"]


In [ ]:
retards_df.columns


In [11]:
# Preprocess data

train_df_X, train_df_Y, test_df_X, test_df_Y  = preprocessing_data("data/regularite-mensuelle-tgv-aqst.csv")


In [13]:
# Import random forest


,duree_moyenne,nb_train_prevu,nb_train_depart_retard,nb_train_retard_arrivee,gare_depart_AIX EN PROVENCE TGV,gare_depart_ANGERS SAINT LAUD,gare_depart_ANGOULEME,gare_depart_ANNECY,gare_depart_ARRAS,gare_depart_AVIGNON TGV,...,gare_arrivee_STUTTGART,gare_arrivee_TOULON,gare_arrivee_TOULOUSE MATABIAU,gare_arrivee_TOURCOING,gare_arrivee_TOURS,gare_arrivee_VALENCE ALIXAN TGV,gare_arrivee_VANNES,gare_arrivee_ZURICH,service_International,service_National
0,-0.032732,0.798606,1.986802,0.352668,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
1,1.569366,-0.260052,-0.735061,-0.454665,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,1.535035,-0.375123,-0.845706,-0.791054,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,2.175874,-1.203638,-0.635481,-1.060164,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,-0.879555,0.171466,-0.403126,-0.151915,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [14]:
# Random forest

from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators = 1000, random_state =42)

rf.fit(train_df_X,train_df_Y)

RandomForestRegressor(n_estimators=1000, random_state=42)

In [18]:

from sklearn.metrics import mean_squared_error, r2_score


# Data prediction

predictions = rf.predict(test_df_X)
predictions_df = pd.DataFrame(predictions, columns=test_df_Y.columns)


for column in test_df_Y.columns:


    mse = mean_squared_error(test_df_Y[column], predictions_df[column])
    r2 = r2_score(test_df_Y[column], predictions_df[column])

    print("Column: %s" % column)
    print("Mean Squared Error:", mse)
    print("R-squared:", r2)


Column: prct_cause_externe
Mean Squared Error: 0.8694284624361069
R-squared: 0.13943724322208517
Column: prct_cause_infra
Mean Squared Error: 0.8415802833494581
R-squared: 0.12761889310207852
Column: prct_cause_gestion_trafic
Mean Squared Error: 0.8004876460123054
R-squared: 0.20228315645592687
Column: prct_cause_materiel_roulant
Mean Squared Error: 0.8542277199303995
R-squared: 0.19613988430562979
Column: prct_cause_gestion_gare
Mean Squared Error: 0.8708580224455708
R-squared: -0.04403937475183284
Column: prct_cause_prise_en_charge_voyageurs
Mean Squared Error: 0.8698152565871484
R-squared: 0.20721511023309236
Column: total_retard_mean
Mean Squared Error: 0.6410209058492702
R-squared: 0.37300390018691276
Column: total_retard_alltrains_mean
Mean Squared Error: 1.0548483628613048
R-squared: 0.09456877547284104


In [ ]:
df_retard_mean = train_df_X.copy()

df_retard_mean["total_retard_mean"] = train_df_Y["total_retard_mean"]

In [ ]:
train_df_Y.head()


In [6]:
from pycaret.regression import *
exp_reg = setup(data=train_df_X.join(train_df_Y["total_retard_mean"]), target='total_retard_mean', session_id=42)

best_model_target1 = compare_models()

# Tune the best model for target1
tuned_model_target1 = tune_model(best_model_target1)

# Predict target1

predictions_target1 = predict_model(tuned_model_target1, data=test_df_X)



,Description,Value
0,Session id,42
1,Target,total_retard_mean
2,Target type,Regression
3,Original data shape,"(5942, 125)"
4,Transformed data shape,"(5942, 125)"
5,Transformed train set shape,"(4159, 125)"
6,Transformed test set shape,"(1783, 125)"
7,Numeric features,124
8,Preprocess,True
9,Imputation type,simple


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.5116,0.6824,0.8183,0.3078,0.3251,3.2206,0.1270
gbr,Gradient Boosting Regressor,0.5194,0.6950,0.8249,0.2977,0.3403,3.0898,0.2200
rf,Random Forest Regressor,0.5127,0.7078,0.8346,0.2703,0.3294,3.1335,0.4700
et,Extra Trees Regressor,0.5326,0.7442,0.8565,0.2366,0.3351,3.3367,0.4110
br,Bayesian Ridge,0.5473,0.7691,0.8686,0.2221,0.3587,2.9390,0.0280
ridge,Ridge Regression,0.5487,0.7706,0.8694,0.2203,0.3553,3.1403,0.0150
xgboost,Extreme Gradient Boosting,0.5331,0.7655,0.8663,0.2094,0.3372,3.3704,0.5630
huber,Huber Regressor,0.5314,0.7878,0.8779,0.2063,0.3591,2.8748,0.2020
knn,K Neighbors Regressor,0.5518,0.7834,0.8785,0.2002,0.3436,3.2543,0.0290
lr,Linear Regression,0.5690,0.7998,0.8837,0.1941,0.3600,3.4852,0.8600


Processing:   0%|          | 0/81 [00:00<?, ?it/s]

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.5044,0.5179,0.7197,0.3014,0.3243,2.9311
1,0.5629,1.1484,1.0716,0.1708,0.3329,3.3338
2,0.5306,0.6301,0.7938,0.2662,0.3451,1.9993
3,0.5351,0.6739,0.8209,0.3440,0.3455,1.9843
4,0.5473,0.9173,0.9577,0.2543,0.3341,4.3832
5,0.5070,0.5053,0.7109,0.3778,0.3179,3.1795
6,0.5029,0.7816,0.8841,0.3002,0.3304,6.3756
7,0.5089,0.4944,0.7031,0.3151,0.3180,1.6990
8,0.5360,0.6910,0.8313,0.3183,0.3457,3.1690


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


In [ ]:
pd.concat([train_df_X, train_df_Y["total_retard_alltrains_mean"]], axis=0, ignore_index=True)

In [ ]:
train_df_Y["total_retard_alltrains_mean"]

In [ ]:
# Blending 

top_models = compare_models(n_select=3)  # You can select any number of top models based on your preference

# Blend the top models
blender = blend_models(estimator_list=top_models)

# Evaluate the blended model
evaluate_model(blender)



In [ ]:
# Stacking 

best_model = compare_models()

top_models = compare_models(n_select=3)

# Stack the top models
stacker = stack_models(estimator_list=top_models, meta_model=best_model)  
# 'best_model' is the model you want to use as the meta-model for stacking

# Evaluate the stacked model
evaluate_model(stacker)

In [20]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

# Define the parameter grid to search
param_grid = {
    'n_estimators': [300, 500, 750],  # Number of trees in the forest
    'max_features': ['auto', 'sqrt', 'log2'],  # Number of features to consider at every split
    'max_depth': [None, 40,50],  # Maximum number of levels in the tree
    'min_samples_split': [2, 5, 8],  # Minimum number of samples required to split an internal node
    'min_samples_leaf': [1, 2, 4]  # Minimum number of samples required to be at a leaf node
}

# Create a RandomForestRegressor
rf = RandomForestRegressor(random_state=42)

# Instantiate GridSearchCV
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)

# Perform the grid search
grid_search.fit(train_df_X, train_df_Y["total_retard_mean"])

# Get the best parameters
best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)

# Train the model with the best hyperparameters
best_rf = RandomForestRegressor(random_state=42, **best_params)
best_rf.fit(train_df_X, train_df_Y["total_retard_mean"])

# Make predictions
predictions = best_rf.predict(test_df_X)


"""
Fitting 5 folds for each of 324 candidates, totalling 1620 fits
Best Hyperparameters: {'max_depth': 30, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 300}

Bueno 2 
Best Hyperparameters: {'max_depth': 40, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 500}
"""
""

Fitting 5 folds for each of 243 candidates, totalling 1215 fits


ValueError: min_samples_split must be an integer greater than 1 or a float in (0.0, 1.0]; got the integer 1

In [9]:
from sklearn.metrics import mean_squared_error, r2_score

# Get the best estimator from the grid search
best_rf = grid_search.best_estimator_

# Make predictions on the test data
predictions = best_rf.predict(test_df_X)

# Calculate and print evaluation metrics
mse = mean_squared_error(test_df_Y["total_retard_mean"], predictions)
r2 = r2_score(test_df_Y["total_retard_mean"], predictions)

print("Mean Squared Error:", mse)
print("R-squared:", r2)




Mean Squared Error: 0.6625293883810572
R-squared: 0.35023844566342466


In [ ]:

from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMRegressor

# Sample data (replace this with your actual data)


# Define the hyperparameters grid to search
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.5],
    'max_depth': [3, 4, 5],
    'num_leaves':[10,20,30]
}

# Create LGBMRegressor model
lgbm = LGBMRegressor(random_state=42)

# Instantiate GridSearchCV
grid_search = GridSearchCV(estimator=lgbm, param_grid=param_grid, cv=5, n_jobs=-1, verbose = 2)

# Perform the grid search
grid_search.fit(train_df_X, train_df_Y["total_retard_mean"])

# Get the best parameters
best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)

# Train the model with the best hyperparameters
best_lgbm = LGBMRegressor(random_state=42, **best_params)
best_lgbm.fit(train_df_X, train_df_Y["total_retard_mean"])

# Now you can use the best_lgbm model for predictions


In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

#Make predictions on the test data
predictions = best_lgbm.predict(test_df_X)

# Calculate evaluation metrics
mae = mean_absolute_error(test_df_Y["total_retard_mean"], predictions)
mse = mean_squared_error(test_df_Y["total_retard_mean"], predictions)
r2 = r2_score(test_df_Y["total_retard_mean"], predictions)

# Print the metrics
print("Mean Absolute Error (MAE):", mae)
print("Mean Squared Error (MSE):", mse)
print("R-squared (R2):", r2)